# Evaluation

Network inspection, learning curves, metrics, and grasp trials.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.chdir('..')

In [ ]:
from __future__ import print_function, division

from pathlib2 import Path
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rospy

from vgn.baselines import GPD
from vgn import benchmark, vis
from vgn.dataset import Dataset
from vgn.detection import *
from vgn.grasp import *
from vgn.utils.transform import Rotation, Transform

In [ ]:
vis.set_size(0.3)

In [ ]:
rospy.init_node("vgn_evaluation", anonymous=True)

Model to be evaluated.

In [ ]:
model_path = Path("")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = load_network(model_path, device)

## Data Set

Visualise detections on some data set.

In [ ]:
dataset = Path("data/datasets/train")
dataset = Dataset(dataset, reconstruction="complete")

Draw all detections of a random test sample.

In [ ]:
i = np.random.choice(len(dataset))
print("Sample:", i)

x, y, index = dataset[i]
vis.draw_sample(x, y, index)
vis.draw_points(np.asarray(dataset.read_pc(i).points))

out = predict(x, net, device)
out = process(x, out, threshold=0.8)
vis.draw_quality(out[0])

grasps, scores = select(out)
grasps = [from_voxel_coordinates(g, vis.VOXEL_SIZE) for g in grasps]
vis.draw_grasps(grasps, scores)

Draw the ground truth and prediction of a random test sample.

In [ ]:
i = np.random.choice(len(dataset))
print("Sample:", i)

x, y, index = dataset[i]
vis.draw_sample(x, y, index)
vis.draw_points(np.asarray(dataset.read_pc(i).points))

out = predict(x, net, device)
grasp, score = select_index(out, index)
grasp = from_voxel_coordinates(grasp, vis.VOXEL_SIZE)
vis.draw_grasps([grasp], [score])

## Benchmark

Metrics and failure cases of a clutter removal experiment.

In [ ]:
logdir = Path("data/experiments/meh")

Compute metrics.

In [ ]:
n_grasps, success_rate, percent_cleared, planning_time = benchmark.compute_metrics(logdir)

print("# grasps:       ", n_grasps)
print("Success rate:   ", success_rate)
print("Percent cleared:", percent_cleared)
print("Planning time:   {} +- {}".format(planning_time["mean"], planning_time["std"]))

Visualize failure cases in rviz.

In [ ]:
dataset = Dataset(logdir)
failures = dataset.df[dataset.df["label"] == 0].index.tolist()
iterator = iter(failures)

In [ ]:
# visualize one by one
i = next(iterator)
print(i)
dataset.draw(i, 0.05)

In [ ]:
# iterate with some time delay
for i in iterator:
    dataset.draw(i, 0.05)
    time.sleep(2.0)

## Performance vs Amount of Scene Information

Evaluate grasp performance with increasingly complete scene reconstructions.

In [ ]:
root = Path("data/experiments/scene_information")
N = 12

In [ ]:
# grasp_planner = VGN(model_path)
grasp_planner = GPD()

In [ ]:
for n in range(1,N):
    benchmark.run(
        grasp_planner,
        root / "gpd" / str(n).zfill(2),
        object_set="test",
        object_count=5,
        rounds=40,
        n=n,
        N=N,
        sim_gui=True,
    )

Plot performance vs number of viewpoints.

In [ ]:
viewpoints = np.arange(1,N)
success_rates = {"vgn" : [], "gpd": []}
percent_cleared = {"vgn": [], "gpd": []}

for n in viewpoints:
    for method in ["vgn", "gpd"]:
        res = benchmark.compute_metrics(root / method / str(n).zfill(2))
        success_rates[method].append(res[1])
        percent_cleared[method].append(res[2])


In [ ]:
fig, ax = plt.subplots()

ax.plot(viewpoints, success_rates["vgn"], color="C0", label="VGN")
ax.plot(viewpoints, percent_cleared["vgn"], color="C0", linestyle="--")

ax.plot(viewpoints, success_rates["gpd"], color="C1", label="GPD")
ax.plot(viewpoints, percent_cleared["gpd"], color="C1", linestyle="--")

ax.set_ylim(0, 101)
ax.legend()